# GOAL: user only needs to modify config

In [1]:
config = {
    'domain': 'stub',
    'postgres': True,
}

In [2]:
# Get DB connection string and add to globals
# NOTE: $SNORKELDB must be set before any snorkel imports
import os

DB_NAME = getattr(config, 'db_name', 'babble_' + config['domain'])
if config['postgres']:
    DB_TYPE = 'postgres'
else:
    DB_TYPE = 'sqlite'
    DB_NAME += '.db'
DB_ADDR = "localhost:{0}".format(config['db_port']) if 'db_port' in config else ""
os.environ['SNORKELDB'] = '{0}://{1}/{2}'.format(DB_TYPE, DB_ADDR, DB_NAME)
print("$SNORKELDB = {0}".format(os.environ['SNORKELDB']))

$SNORKELDB = postgres:///babble_stub


In [3]:
from snorkel import SnorkelSession
session = SnorkelSession()

# Resolve config conflicts (nb_config > local_config > global_config)
from snorkel.contrib.babble.pipelines import (
    global_config, get_local_config, get_local_pipeline, recursive_merge_dicts)

local_config = get_local_config(config['domain'])
config = recursive_merge_dicts(local_config, config)
config = recursive_merge_dicts(global_config, config)


from snorkel.models import candidate_subclass
candidate_class = candidate_subclass(config['candidate_name'], 
                                     config['candidate_entities'])

pipeline = get_local_pipeline(config['domain'])
pipe = pipeline(session, candidate_class, config)

In [4]:
pipe.parse()

I parsed!


In [5]:
pipe.extract()

I extracted!


In [6]:
pipe.load_gold()

I loaded gold!


In [7]:
pipe.collect()

I collected signal!


In [8]:
pipe.label()

I labeled!


In [9]:
pipe.supervise()

I supervised!


In [10]:
pipe.classify()

I classified!
